# Dataaugmentation
## Dieses Notebook soll den Pascal VOC Datensatz duch Data Augmentation erweitern, sodass eventuell neue Features gelernt werden können.

In [2]:
import cv2 as cv
import os
import numpy as np
from PIL import Image
import time
from multiprocessing import Process

Es wird eine Klasse zur Verfügung gestellt, die Raw und Colormap Label Images drei mal im Uhrzeigersinn um 90° rotieren lässt und anschließend flipped und dann wieder im Uhrzeigersinn um 90° rotieren lässt. 

Wenn nötig, werden die Colormap Label verschlüsselt und ebenfalls rotiert und geflipped. Dies wird jedoch nur getan, wenn ein gültiger Ausgabepfad für die Verschlüsselung vorliegt.

Die verarbeiteten Bilder werden aufsteigend nummeriert und die Dataaugmenatation wird deutlich gemacht durch einen unterstrich und weiterer aufsteigender Nummerierung. Das bedeutet die erste Datei hat den Namen 0.png und die erste Dataaugmentation hat den namen 0_0.png. Die zweite Datei mit der dazugehörigen Augmentation wäre 1.png und 1_0.png

In [9]:
class augmentation:
    def __init__(self, filename_input, filename_output, path_Input_raw, path_output_raw, 
                   path_input_ss, path_output_ss, imgFormat_raw, imgFormat_ss,
                   path_output_ss_enc=None, count=0):
        #augList = readFile("Segmentation/test.txt")
        #convert("JPEGImages/", "new_Dataset/raw/", augList, "jpg")
        self.count = count
        self.fileListe = open(filename_output, "w")
        self.i = self.count
        
        augList = self._readFile(filename_input)
        
        # Dataaugmentation nur auf den Raw Images
        self._convert(path_Input_raw, path_output_raw, augList, imgFormat_raw)
        self.i = self.count
        
        # Wenn Encode Pfad angegeben, dann Encode und mache Dataaugmentation
        if(path_output_ss_enc is not None):
            start = time.time()
            self._convertss(path_input_ss, path_output_ss_enc, augList, imgFormat_ss)
            ende = time.time()
            #print('{:5.3f}s'.format(ende-start))
        self.i = self.count
        
        # Dataaugmenatation auf den Colormap Label
        self._convert(path_input_ss, path_output_ss, augList, imgFormat_ss)
    
    def __del__(self):
        self.fileListe.close()
    
    def _readFile(self, filename):
        augList = []
        file = open(filename, "r")
        for line in file:
            augList.append(line[0:len(line)-1]) #String Handling wird hier benötigt, um den Zeilenumbruch zu entfernen
        file.close()
        return augList
    
    def _convert(self, path_Input, path_output, augList, imgFormat):
        j = 0
        for imgName in augList:
            img = cv.imread(path_Input + imgName + "." + imgFormat)
            
            j = self._imgRotate(img, path_output, imgFormat, j)
            j = self._flipRotate(img, path_output, imgFormat, j)
            cv.imwrite(path_output + str(self.i) + "." + imgFormat, img)
            self.fileListe.write(str(self.i) + "\n")
            self.i += 1
            j = 0
    
    def _convertss(self, path_Input, path_output, augList, imgFormat):
        j = 0
        for imgName in augList:
            img = cv.imread(path_Input + imgName + "." + imgFormat)
            
            img = self.encodeImg(img)
            
            j = self._imgRotatess(img, path_output, imgFormat, j)
            j = self._flipRotatess(img, path_output, imgFormat, j)
    
            cv.imwrite(path_output + str(self.i) + "." + imgFormat, img)
            
            #self.fileListe.write(str(self.i) + "\n")
            self.i += 1
            j = 0
            
    def _flipRotatess(self, img, path_output, imgFormat, j):
        img = cv.flip(img, 1)
        #img = self.encodeImg(img)
        cv.imwrite(path_output + str(self.i) + "_" + str(j) + "." + imgFormat, img)
        #self.fileListe.write(str(self.i) + "_" + str(j) + "\n")
        j += 1
        j = self._imgRotate(img, path_output, imgFormat, j)
        return j
    
    def _imgRotatess(self, img, path_output, imgFormat, j):
        for x in range (0, 3):
            img = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
            #img = self.encodeImg(img)
            cv.imwrite(path_output + str(self.i) + "_" + str(j) + "." + imgFormat, img)
            #self.fileListe.write(str(self.i) + "_" + str(j) + "\n")
            j += 1
        return j

    def _flipRotate(self, img, path_output, imgFormat, j):
        img = cv.flip(img, 1)
        cv.imwrite(path_output + str(self.i) + "_" + str(j) + "." + imgFormat, img)
        self.fileListe.write(str(self.i) + "_" + str(j) + "\n")
        j += 1
        j = self._imgRotate(img, path_output, imgFormat, j)
        return j
    
    def _imgRotate(self, img, path_output, imgFormat, j):
        for x in range (0, 3):
            img = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
            cv.imwrite(path_output + str(self.i) + "_" + str(j) + "." + imgFormat, img)
            self.fileListe.write(str(self.i) + "_" + str(j) + "\n")
            j += 1
        return j
    
    def getNumber(self):
        return self.i
    
    def convertcmap(self, cmap):
        label = {}
        i = 0
        for k in cmap:
            #print(k)
            new = {i : k}
            label.update(new)
            i +=1
        return label

    def newmsk(self, msk, cmap):
        new_msk = np.zeros((msk.shape[0], msk.shape[1]), dtype=np.uint8)
        xx = 0
        yy = 0
        count = 0
        for x in msk:
            for y in x:
                isThere=False
                for key in cmap:
                    #print(np.array_equal(y, cmap[key]))
                    #print(y)
                    #print(cmap[key])
                    #print(key)
                    if(np.array_equal(y, cmap[key])):
                        new_msk[xx][yy] = key
                        isThere = True
                if(not isThere):
                    count += 1
                    new_msk[xx][yy] = 255
                yy += 1
            yy = 0
            xx += 1
        return new_msk, count
    
    def encodeImg(self, img):
        cmap = np.load('./cmap.npy')
        #img_path = load_images_from_folder(pathIn)
        cmap = self.convertcmap(cmap)
        img, count = self.newmsk(img, cmap)
        return img
        '''for path in img_path:
            print(path.split('/')[2])
            img = np.array(Image.open(path))
            img, count = newmsk(img, cmap)
            img = Image.fromarray(img)
            img.save(pathOut + path.split('/')[2])'''

        

In der unteren Zeile wird die obere Klasse erzeugt. Dabei werden alle Images für das Training verarbeitet. Dies geschieht über die Datei "trainval.txt". Diese Datei enthält die Namen für alle Bilder die für das Training mit dem Pascal Voc Datensatz verwendet werden.

In [10]:
filename_input = "Segmentation/trainval.txt"
filename_output = "new_Dataset/list_new/bla.txt" # Wird nicht mehr benötigt!
path_Input_raw = "JPEGImages/"
path_output_raw = "new_Dataset/raw/"
path_input_ss = "SegmentationClass/"
path_output_ss = "new_Dataset/semantic_new/"
path_output_ss_enc = "new_Dataset/semantic_enc/"
imgFormat_raw = "jpg"
imgFormat_ss = "png"
aug = augmentation(filename_input, filename_output, path_Input_raw, path_output_raw, 
                   path_input_ss, path_output_ss, imgFormat_raw, imgFormat_ss, path_output_ss_enc)
count = aug.getNumber()
del(aug)

Es wurde eine weitere Funktion entwickelt, die die Datei "trainval.txt" verarbeitet. Dabei wird diese Datei mit der "train.txt" und der "val.txt" verglichen und es werden neue trainings- und validierungstext, passend zu den erstellten Daten bei der Dataaugmentation, generiert.

In [36]:
def createFile(fileTrainVal, fileTrain, fileVal, fileOutTrain, fileOutVal, path_raw, path_semantic):
        fileTrainVal = open(fileTrainVal, "r")
        fileTrain = open(fileTrain, "r")
        fileVal = open(fileVal, "r")
        
        arrayTrainVal = []
        arrayTrain = []
        arrayVal = []
        
        fileOutTrain = open(fileOutTrain, "w")
        fileOutVal = open(fileOutVal, "w")
        
        
        for line in fileTrainVal:
            txt = line[0:len(line)-1] #String Handling wird hier benötigt, um den Zeilenumbruch zu entfernen
            arrayTrainVal.append(txt)
        for line in fileTrain:
            txt = line[0:len(line)-1] #String Handling wird hier benötigt, um den Zeilenumbruch zu entfernen
            arrayTrain.append(txt)
        for line in fileVal:
            txt = line[0:len(line)-1] #String Handling wird hier benötigt, um den Zeilenumbruch zu entfernen
            arrayVal.append(txt)
        
        fileTrainVal.close()
        fileTrain.close()
        fileVal.close()
        i = 0
        for trainVal in arrayTrainVal:
            found = False
            
            for train in arrayTrain:
                if(trainVal == train):
                    #print("{}:{}: {}".format(trainVal, train, i))
                    txt = str(i)
                    fileOutTrain.write(path_raw + txt + ".jpg" + " " + path_semantic + txt + ".png" + '\n')
                    for j in range(0,7):
                        txt = "{}_{}".format(i,j)
                        fileOutTrain.write(path_raw + txt + ".jpg" + " " + path_semantic + txt + ".png" + '\n')
                    found = True
                    break
            if(not found):
                for val in arrayVal:
                    if(trainVal == val):
                        txt = str(i)
                        fileOutVal.write(path_raw + txt + ".jpg" + " " + path_semantic + txt + ".png" + '\n')
                        for j in range(0,7):
                            txt = "{}_{}".format(i,j)
                            fileOutVal.write(path_raw + txt + ".jpg" + " " + path_semantic + txt + ".png" + '\n')
                        break
            i += 1
            #file1.write(path_raw + txt + ".jpg" + " " + path_seg + txt + ".png" + '\n')
        print(i)
        fileOutTrain.close()
        fileOutVal.close()
        
fileTrainVal = "Segmentation/trainval.txt"
fileTrain = "Segmentation/train.txt"
fileVal = "Segmentation/val.txt"
fileOutTrain = 'new_Dataset/list/train.txt'
fileOutVal = 'new_Dataset/list/val.txt'
path_raw = "raw/"
path_semantic = "semantic/"

createFile(fileTrainVal, fileTrain, fileVal, fileOutTrain, fileOutVal, path_raw, path_semantic)

2913
